# HTTP Request/Response Cycle - Lab

## Introduction 

In this lab, we'll make use of the `requests` module commands and properties seen in the previous lesson, to extract information for a web service called **"Open Notify"** to access NASA's space data. 

## Objectives

You will be able to:

* Explain the HTTP request/response cycle
* List the status codes of responses and their meanings
* Obtain and interpret status codes from responses
* Make HTTP GET and POST requests in python using the `requests` library

## Open Notify 

[Open Notify](http://open-notify.org/)  is an an open source project to provide a simple programming interface for some of NASA’s awesome data. This takes live raw data from NASA's systems and turn them into APIs related to space and spacecraft. We can access the following information from open notify. 

* Current Location of the International Space Station

* Overhead Pass Predictions for the International Space Station

* Number of People in Space
    
### API endpoints

Open Notify has several API endpoints. 
>An endpoint is a server route that is used to retrieve different data from the API. 

For example, the `/comments` endpoint on the Reddit API might retrieve information about comments, whereas the `/users` endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

For the OpenNotify API, we have following end points 

1. Current Location of the International Space Station `/iss-now.json`
2. Overhead Pass Predictions for the International Space Station `/iss-pass.json`    
3. Number of People in Space `/astros.json`

The `.json` extension simply tells us that the data is being returned in a JSON format.

In this lab, we'll be querying this API to retrieve live data about the International Space Station (ISS). Details on OpenNofity, endpoints, syntax, and the services it offers can be viewed [Here](http://open-notify.org/Open-Notify-API/)

![](images/iss.jpg)

### Current location of International Space Station

The first endpoint we'll look at on Open Notify is the ` iss-now.json` endpoint (current location of international space station). This endpoint gets the current latitude and longitude of the International Space Station.  Perform the following tasks 
* Make a get request to get the latest position of the international space station from the opennotify api's `iss-now` endpoint at http://api.open-notify.org/iss-now.json
* Check the status code of the response
* Interpret the returned code

In [1]:
# Your Code Here
import requests

r = requests.get('http://api.open-notify.org/iss-now.json')
resp = r.json()
print(r.status_code == requests.codes.ok)
print(r.text)

True
{"iss_position": {"latitude": "3.6109", "longitude": "77.1534"}, "message": "success", "timestamp": 1607909039}


In [2]:
# Your comments 
# The request was a success, both latitude and longitude have been returned

* Print the contents of the response and identify its current location

In [3]:
# Your Code Here
iss_lat = resp['iss_position']['latitude']
iss_long = resp['iss_position']['longitude']
print('Latitude:', iss_lat, '| Longitude:', iss_long)

Latitude: 3.6109 | Longitude: 77.1534


In [ ]:
# !pip install reverse_geocoder

In [15]:
# Interpret your results using the API
import reverse_geocoder as rg
from datetime import datetime

coordinates = iss_lat, iss_long
location = rg.search(coordinates)
city = location[0]['name']
country = location[0]['cc']
time = resp['timestamp']
dt = datetime.fromtimestamp(time)
print('As of', dt ,'UTC, the ISS is passing {}, {}.'.format(city, country))

As of 2020-12-13 20:23:59 UTC, the ISS is passing Felidhoo, MV.


### Check the next pass of International space station for a given location

Let's repeat the above for the second endpoint `iss-pass.json`. This end point is used to query the next pass of the space station on a given location. Let's just run as above and record your observations.

In [7]:
# Your Code Here
r2 = requests.get('http://api.open-notify.org/iss-pass.json')
resp2 = r2.json()
print(r2.status_code == requests.codes.ok)
print(r2.text)

False
{
  "message": "failure", 
  "reason": "Latitude must be specified"
}



In [8]:
# Your comments 
# This request is a failure, and the reason being that latitude must be specified

So clearly there is something wrong as we had a 400 response. This is how you should always test your responses for validity. 

If we look at the documentation for the Open Notify API, we see that the ISS Pass endpoint requires two parameters.

> The ISS Pass endpoint returns when the ISS will next pass over a given location on earth. In order to compute this, we need to pass the coordinates of the location to the API. We do this by passing two parameters -- latitude and longitude.

We can do this by adding an optional keyword argument, `params`, to our request. In this case, there are two parameters we need to pass:

* `lat` -- The latitude of the location we want.
* `lon` -- The longitude of the location we want.

Perform the following tasks :
* Set parameters to reflect the lat and long of New York  (40.71, -74)
* Send a get request to Open Notify passing in the lat long parameters as k:v pairs in a dictionary
* Check the status code and interpret
* Print the header information and the returned content

In [9]:
# Your Code Here
coordinates = {'lat': '40.71', 'lon': '-74'}
r2 = requests.get('http://api.open-notify.org/iss-pass.json', params=coordinates)
resp2 = r2.json()
print(r2.status_code == requests.codes.ok)
print(r2.text)

True
{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1607905624, 
    "latitude": 40.71, 
    "longitude": -74.0, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 531, 
      "risetime": 1607948390
    }, 
    {
      "duration": 654, 
      "risetime": 1607954109
    }, 
    {
      "duration": 599, 
      "risetime": 1607959969
    }, 
    {
      "duration": 560, 
      "risetime": 1607965857
    }, 
    {
      "duration": 621, 
      "risetime": 1607971688
    }
  ]
}



In [10]:
# Check the API and interpret your results - when will ISS pass over NEW York next ?
n = resp2['request']['passes']
next_ts = resp2['response'][0]['risetime']
next_dt = datetime.fromtimestamp(next_ts)
duration = resp2['response'][0]['duration']
print('The next time the ISS passes New York is', next_dt, 'UTC, for', duration, 'seconds or', duration/60, 'minutes')

The next time the ISS passes New York is 2020-12-14 07:19:50 UTC, for 531 seconds or 8.85 minutes


### Finding the number of people in space

Open Notify has one more API endpoint, `/astros.json`. It tells you how many people are currently in space. The format of the responses can be studied [HERE](http://open-notify.org/Open-Notify-API/People-In-Space/).

Read the above documentation and perform the following tasks:

* Get the response from astros.json endpoint
* Count how many people are currently in space
* List the names of people currently in space.

In [11]:
# Your Code Here
r3 = requests.get('http://api.open-notify.org/astros.json')
resp3 = r3.json()
print(r3.status_code == requests.codes.ok)
print(r3.text)

True
{"message": "success", "number": 7, "people": [{"craft": "ISS", "name": "Sergey Ryzhikov"}, {"craft": "ISS", "name": "Kate Rubins"}, {"craft": "ISS", "name": "Sergey Kud-Sverchkov"}, {"craft": "ISS", "name": "Mike Hopkins"}, {"craft": "ISS", "name": "Victor Glover"}, {"craft": "ISS", "name": "Shannon Walker"}, {"craft": "ISS", "name": "Soichi Noguchi"}]}


In [12]:
# Interpret the Results - How many people are in space and what are their names 
number = resp3['number']
people = [(x['name'], x['craft']) for x in resp3['people']] 

print('There are currently', number, 'people in space, and their names are:')
for name, craft in people:
    print('-', name, '-', craft)

There are currently 7 people in space, and their names are:
- Sergey Ryzhikov - ISS
- Kate Rubins - ISS
- Sergey Kud-Sverchkov - ISS
- Mike Hopkins - ISS
- Victor Glover - ISS
- Shannon Walker - ISS
- Soichi Noguchi - ISS


## Summary 

In this lesson, we saw how we can use request and response methods to query an Open API. We also saw how to look at the contents returned with the API calls and how to parse them. Next, we'll look at connecting to APIs which are not OPEN, i.e. we would need to pass in some authentication information and filter the results. 